In [26]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

# Load dataset
file_path = 'C:/Users/asus/Desktop/E_Commerce_Recommedation_System/ratings_Beauty.csv'  # Update path if needed
df = pd.read_csv(file_path, names=["UserID", "ProductID", "Rating", "Timestamp"], skiprows=1)

# Drop unnecessary columns
df.drop(columns=["Timestamp"], inplace=True)

# Convert Rating to numeric & remove invalid ratings
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")
df = df.dropna(subset=["Rating"])

# Create mappings for UserID and ProductID
user_map = {user: i for i, user in enumerate(df["UserID"].unique())}
product_map = {product: i for i, product in enumerate(df["ProductID"].unique())}

# Convert original IDs to numerical indices
df["user_index"] = df["UserID"].map(user_map)
df["product_index"] = df["ProductID"].map(product_map)

# Create a sparse matrix (User-Product Interaction)
sparse_matrix = csr_matrix(
    (df["Rating"], (df["user_index"], df["product_index"])), 
    shape=(len(user_map), len(product_map))
)

# Train KNN model (Collaborative Filtering)
knn = NearestNeighbors(metric="cosine", algorithm="brute")
knn.fit(sparse_matrix)

# Reverse mapping for ProductID lookup
reverse_product_map = {i: product for product, i in product_map.items()}

# Find most popular products (for fallback)
popular_products = df.groupby("ProductID")["Rating"].count().sort_values(ascending=False).index[:5].tolist()

# Function to recommend products in real-time
def recommend_real_time(user_id, top_n=5):
    if user_id not in user_map:
        return "User not found."

    user_index = user_map[user_id]

    # Find nearest neighbors
    distances, indices = knn.kneighbors(sparse_matrix[user_index], n_neighbors=top_n + 1)

    # Get recommended product indices (excluding the user itself)
    recommended_indices = indices.flatten()[1:]

    # Convert numerical indices back to original ProductIDs, ensuring validity
    recommended_products = [reverse_product_map[idx] for idx in recommended_indices if idx in reverse_product_map]

    # Fallback to popular products if no valid recommendations
    if not recommended_products:
        recommended_products = popular_products

    return recommended_products

# Example Usage
sample_user_id = df["UserID"].iloc[0]  # Pick a random user
recommended_products = recommend_real_time(sample_user_id)

print(f"Real-Time Recommendations for User {sample_user_id}: {recommended_products}")


Real-Time Recommendations for User A39HTATAQ9V7YF: ['B001MA0QY2', 'B0009V1YR8', 'B0043OYFKU', 'B0000YUXI0', 'B003V265QW']
